In [ ]:
#python DLTools/SetupPython.py > mysetup.py
import mysetup
from LArTPCDNN.LoadData import *

n_threads=1
n_threads2=1
multiplier=1
BatchSize=16
FileSearch="/data/LArIAT/h5_files/*.h5"

DownSampleSize=8
ScanWindowSize=256
Normalize=True
closefiles=False

NSamples=10000
EnergyCut=0.61

Particles= ['electron', 'antielectron',
            'pion',             
            'photon',
            'pionPlus', 'pionMinus',
            'proton', 'antiproton',
            'muon', 'antimuon',
            'kaonMinus', 'kaonPlus']

NClasses=len(Particles)
if ScanWindowSize>0:
#    shapes=[(BatchSize*multiplier, 2, 240, ScanWindowSize), (BatchSize*multiplier, NClasses)]
    shapes=[(BatchSize*multiplier, 240, ScanWindowSize),
            (BatchSize*multiplier, 240, ScanWindowSize),
            (BatchSize*multiplier, NClasses)]
    viewshape=(None, 240, ScanWindowSize)
else:
    shapes=[(BatchSize*multiplier, 240, 4096/DownSampleSize),
            (BatchSize*multiplier, 240, 4096/DownSampleSize),
            (BatchSize*multiplier, NClasses)]

    viewshape=(None, 240, 4096/DownSampleSize)


TrainSampleList,TestSampleList=DivideFiles(FileSearch,[.9,.1],
                                           datasetnames=[u'features'],
                                           Particles=Particles)

def MakeGenerator(SampleList,NSamples,
                  cachefile="LArIAT-LoadDataTest-Cache.h5",**kwargs):

    return DLMultiClassFilterGenerator(TrainSampleList, FilterEnergy(EnergyCut), max=NSamples,
                                       preprocessfunction=ProcessWireData(DownSampleSize,ScanWindowSize,Normalize),
                                       postprocessfunction=MergeInputs(),
                                       batchsize=BatchSize,
                                       shapes=shapes,
                                       n_threads=n_threads,
                                       multiplier=multiplier,
                                       cachefile=cachefile,
                                       **kwargs)

# Use DLGenerators to read data
Train_gen = MakeGenerator(TrainSampleList, NSamples,
                            cachefile="/tmp/LArTPCDNN-LArIAT-TrainEvent-Cache.h5")

gen=Train_gen.DiskCacheGenerator(n_threads2)

Data=gen.next()

In [ ]:
Data[0][1].shape

In [ ]:
from keras.models import load_model

model = load_model('/home/pjsadows/ml/particles/1.1/trained/model_20_epoch67_loss1.65917.h5')  
